In [10]:
import pandas as pd

In [11]:
df1 = pd.read_csv("new new location with some cols - output.csv")

In [12]:
df2 = pd.read_csv("Nppes data_gastro.csv")

C:\Users\sanja\AppData\Local\Temp\ipykernel_20116\1423183361.py:1: DtypeWarning: Columns (17,18,25,40,41,65,69,73,77,78,81,82,85,86,88,89,90,91,92,94,95,96,97,98,99,100,101,102,103,104,105,106,107,116,119,120,122,123,124,126,127,128,130,131,132,134,135,136,138,139,140,142,143,144,146,147,148,150,151,152,154,155,156,158,159,160,162,163,164,166,167,168,170,171,172,174,175,176,178,179,180,182,183,184,186,187,188,190,191,325,326,327,328,329) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("Nppes data_gastro.csv")


In [13]:
df1.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Name', 'Fullname', 'NPI', 'Link',
       'Specialty', 'Address', 'Phone', 'Website', 'Page',
       'Fuzzy matching percentage', 'City', 'State', 'PostalCode'],
      dtype='object')

In [14]:
df2.columns

Index(['NPI', 'Entity Type Code', 'Replacement NPI',
       'Employer Identification Number (EIN)',
       'Provider Organization Name (Legal Business Name)', 'Fullname',
       'Provider Last Name (Legal Name)', 'Provider First Name',
       'Provider Middle Name', 'Provider Name Prefix Text',
       ...
       'Healthcare Provider Taxonomy Group_7',
       'Healthcare Provider Taxonomy Group_8',
       'Healthcare Provider Taxonomy Group_9',
       'Healthcare Provider Taxonomy Group_10',
       'Healthcare Provider Taxonomy Group_11',
       'Healthcare Provider Taxonomy Group_12',
       'Healthcare Provider Taxonomy Group_13',
       'Healthcare Provider Taxonomy Group_14',
       'Healthcare Provider Taxonomy Group_15', 'Certification Date'],
      dtype='object', length=331)

In [15]:
import pandas as pd
from fuzzywuzzy import process

def fuzzy_match(value, choices, threshold=80):
    """Find the best fuzzy match for a given value from choices within an NPI group."""
    if not choices:  # If no choices available for that NPI id, return None
        return None
    best_match, score = process.extractOne(value, choices)
    return (best_match, score) if score >= threshold else (None, score)


# Initialize new columns for matched values
df1["matched_postalcode"] = None
df1["postalcode_match_score"] = 0
df1['5dig_postalcode'] = None
df1['5dig_match_score'] = 0
df1["matched_city"] = None
df1["city_match_score"] = 0
df1["matched_state"] = None
df1["state_match_score"] = 0

# Perform fuzzy matching within the same NPI id group
for index, row in df1.iterrows():
    npi_id = row["NPI"]
    
    # Filter df2 to only include rows with the same NPI
    df2_subset = df2[df2["NPI"] == npi_id]
    
    # Perform fuzzy matching within this subset
    best_postalcode, postalcode_score = fuzzy_match(
        str(row["PostalCode"]), 
        df2_subset["Provider Business Mailing Address Postal Code"].astype(str).tolist() + df2_subset["Provider Business Practice Location Address Postal Code"].astype(str).tolist()
    )

    df1.at[index, "matched_postalcode"] = best_postalcode
    df1.at[index, "postalcode_match_score"] = postalcode_score

    best_postalcode1, postalcode_score1 = fuzzy_match(
        str(row["PostalCode"]), 
        [str(pc)[:5] for pc in df2_subset["Provider Business Mailing Address Postal Code"].astype(str)[:5].tolist()] + 
        [str(pc)[:5] for pc in df2_subset["Provider Business Practice Location Address Postal Code"].astype(str)[:5].tolist()]
    )

    df1.at[index, "5dig_postalcode"] = best_postalcode1
    df1.at[index, "5dig_match_score"] = postalcode_score1
    
    best_city, city_score = fuzzy_match(
        str(row["City"]), 
        df2_subset["Provider Business Mailing Address City Name"].astype(str).tolist() + df2_subset["Provider Business Practice Location Address City Name"].astype(str).tolist()
    )
    
    df1.at[index, "matched_city"] = best_city
    df1.at[index, "city_match_score"] = city_score

    best_state, state_score = fuzzy_match(
        str(row["State"]), 
        df2_subset["Provider Business Mailing Address State Name"].astype(str).tolist() + df2_subset["Provider Business Practice Location Address State Name"].astype(str).tolist()
    )

    df1.at[index, "matched_state"] = best_state
    df1.at[index, "state_match_score"] = state_score


In [16]:
df1.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Name', 'Fullname', 'NPI', 'Link',
       'Specialty', 'Address', 'Phone', 'Website', 'Page',
       'Fuzzy matching percentage', 'City', 'State', 'PostalCode',
       'matched_postalcode', 'postalcode_match_score', '5dig_postalcode',
       '5dig_match_score', 'matched_city', 'city_match_score', 'matched_state',
       'state_match_score'],
      dtype='object')

In [17]:
df1.to_csv('new output with some cols.csv')

In [18]:

# Merge both dataframes on NPI
df_merged = df1.merge(df2, on="NPI", how="left")

# Save the merged file
df_merged.to_csv("new output with all cols.csv", index=False)
